In [94]:
from datasets import load_dataset
import requests
import ciso8601
import time
import json

In [75]:
ds = load_dataset("codeparrot/github-code", streaming=True, split='train', languages=["Python"])
iterable_ds = iter(ds)

Using custom data configuration Python-all-4b2efe4a27feed92


In [98]:
def construct_feature_set(code_entry) -> dict[str:int]:
    features = dict();
    user_repo_name = code_entry['repo_name'].split('/')
    owner = user_repo_name[0]
    repo = "".join(user_repo_name[1:])
    repo_response = requests.get(f"https://api.github.com/repos/{owner}/{repo}")
    content = json.loads(repo_response.text)
    # features["num_stars"] = content["stargazers_count"]
    features["num_forks"] = content["forks_count"]
    features["num_watchers"] = content["watchers_count"]
    features["num_open_issues"] = content["open_issues_count"]
    parsed_datetime = ciso8601.parse_datetime(content["created_at"])
    timestamp = time.mktime(parsed_datetime.timetuple())
    features["created_at"] = timestamp
    return features

In [90]:
# THE BELOW IS FOR PYTHON FILES (LEVERAGES INDENTATION RULES)
def construct_list_of_functions(raw_code: str) -> list[str]:
    lines = raw_code.split('\n')
    start = -1
    functions = []
    amnt_tabs = 0
    for i in range(len(lines)):
        # disregard empty lines (prune trailing whitespace later)
        if (start != -1 and len(lines[i]) > 0):
            amnt_tabs_new = len(lines[i].rstrip()) - len(lines[i].strip())
            if amnt_tabs_new <= amnt_tabs:
                functions.append(("\n".join(lines[start:i])).strip())
                start = -1
        elif lines[i].lstrip().startswith("def "):
            start = i
            amnt_tabs = len(lines[i].rstrip()) - len(lines[i].strip())
    return functions


In [102]:
MAX_AMNT_CODE_ENTRIES = 1000
augmented_code_entry_data = []
for i in range(MAX_AMNT_CODE_ENTRIES):
    code_entry = next(iterable_ds)
    # features_set = construct_feature_set(code_entry)
    functions = construct_list_of_functions(code_entry["code"])
    data = dict()
    data["features"] = []
    data["snippets"] = functions
    augmented_code_entry_data.append(data)

In [104]:
print(augmented_code_entry_data)

[{'features': [], 'snippets': ['def __init__(\n        self,\n        credential: "AsyncTokenCredential",\n        subscription_id: str,\n        base_url: str = "https://management.azure.com",\n        **kwargs: Any', 'def _send_request(\n        self,\n        request: HttpRequest,\n        **kwargs: Any']}, {'features': [], 'snippets': []}, {'features': [], 'snippets': []}, {'features': [], 'snippets': ['def __init__(self, bot, settings):\r\n\t\tself.bot = bot\r\n\t\tself.settings = settings', 'def getTimeFromOffset(self, offset):\r\n\t\toffset = offset.replace(\'+\', \'\')\r\n\t\t# Split time string by : and get hour/minute values\r\n\t\ttry:\r\n\t\t\thours, minutes = map(int, offset.split(\':\'))\r\n\t\texcept Exception:\r\n\t\t\ttry:\r\n\t\t\t\thours = int(offset)\r\n\t\t\t\tminutes = 0\r\n\t\t\texcept Exception:\r\n\t\t\t\treturn None\r\n\t\t\t\t# await ctx.channel.send(\'Offset has to be in +-H:M!\')\r\n\t\t\t\t# return\r\n\t\tmsg = \'UTC\'\r\n\t\t# Get the time\r\n\t\tt = date